In [1]:
import os

PATH = '/home/stadlerm/Development/Projects/DreamChallenge/RA2_alpine_lads/ra_joint_predictions'

os.chdir(PATH)

In [9]:
from utils.config import Config

config = Config()

In [3]:
import numpy as np
import tensorflow as tf

from model.utils.metrics import argmax_rsme, softmax_rsme_metric, class_softmax_rsme_metric

dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(5)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(5), 0)
}

feet_narrowing_model = tf.keras.models.load_model('feet_narrowing_v1.h5', custom_objects=dependencies)
hands_narrowing_model = tf.keras.models.load_model('hands_narrowing_v1.h5', custom_objects=dependencies)

In [4]:
dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(6)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(6), 0)
}

print(dependencies)

hand_erosion_model = tf.keras.models.load_model('hands_erosion_v1.h5', custom_objects=dependencies)

{'softmax_rsme': <function softmax_rsme_metric.<locals>.softmax_rsme at 0x7fda2c792730>, 'argmax_rsme': <function argmax_rsme at 0x7fda16cdfae8>, 'class_softmax_rmse_0': <function class_softmax_rsme_metric.<locals>.class_softmax_rmse at 0x7fda2d0cd0d0>}


In [6]:
dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(11)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(11), 0)
}

print(dependencies)

feet_erosion_model = tf.keras.models.load_model('feet_erosion_v1.h5', custom_objects=dependencies)

{'softmax_rsme': <function softmax_rsme_metric.<locals>.softmax_rsme at 0x7fda2c792158>, 'argmax_rsme': <function argmax_rsme at 0x7fda16cdfae8>, 'class_softmax_rmse_0': <function class_softmax_rsme_metric.<locals>.class_softmax_rmse at 0x7fda2c792598>}


In [8]:
dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(5)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(5), 0)
}

wrists_narrowing_model = tf.keras.models.load_model('wrists_narrowing_v1.h5', custom_objects=dependencies)

In [10]:
dependencies = {
    'softmax_rsme': softmax_rsme_metric(np.arange(6)),
    'argmax_rsme': argmax_rsme,
    'class_softmax_rmse_0': class_softmax_rsme_metric(np.arange(6), 0)
}

wrists_erosion_model = tf.keras.models.load_model('wrists_erosion_v1.h5', custom_objects=dependencies)

In [10]:
import numpy as np
import dataset.joint_dataset as joint_dataset
from dataset.test_dataset import joint_test_dataset

dataset = joint_test_dataset(config, config.train_fixed_location)
feet_test_dataset = dataset.get_feet_joint_test_dataset()

preds = {}

for file_info, img in feet_test_dataset:
    file_info = file_info.numpy()
    
    img_info = file_info[0].decode('utf-8').split('.')[0]
    key = file_info[3].decode('utf-8')
    
    patient_info = img_info.split('-')
    patient_id = patient_info[0]
    part = patient_info[1]
    
    # If patient_id is not found, init with empty struct
    if not patient_id in preds.keys():
        preds[patient_id] = {
            'Patient_ID': patient_id
        }
        
    outcome_key = joint_dataset.foot_outcome_mapping[key][0][0].format(part = part)
        
    y_pred = feet_narrowing_model.predict(tf.expand_dims(img, 0))
    y_pred = np.sum(y_pred * np.arange(5))
    
    preds[patient_id][outcome_key] = y_pred

In [11]:
import pandas as pd

x = pd.DataFrame(preds.values(), index = np.arange(len(preds.values())))

In [20]:
narrowing_mask = ['_J_' in column_name for column_name in x.columns]
narrowing_sum = np.sum(x.iloc[:, narrowing_mask].to_numpy(), axis = 1)

x['total_narrow'] = narrowing_sum

print(x['total_narrow'])

0      0.037263
1      0.150300
2      0.003294
3      0.000114
4      0.051440
5      0.001055
6     14.580669
7      0.005593
8     21.036484
9      2.113368
10     0.001045
11     3.001504
12     5.068290
13    19.595230
14    26.012099
15     0.000518
16     0.007254
17     0.014993
18    34.016105
19     0.000140
20    18.021897
21     0.000251
22     0.002758
23     7.766433
24     0.000030
25     0.002066
26     0.008271
27     0.010099
28     3.001497
29     0.000020
        ...    
62    23.999194
63     0.004326
64    14.672130
65     0.002361
66     0.589079
67     0.002384
68     2.002963
69     0.000647
70     0.000012
71     0.004375
72     3.984000
73     0.010668
74     0.002026
75     2.001597
76     0.758970
77     0.004160
78     2.009284
79     0.033065
80     0.015799
81     0.024574
82     0.209609
83     0.002665
84     2.822544
85     0.000106
86     8.001113
87     0.001527
88     0.002898
89     0.030707
90     3.999832
91     0.006458
Name: total_narrow, Leng

In [19]:
print(np.sum(x.iloc[:, narrowing_mask].to_numpy(), axis = 1))

[3.72632005e-02 1.50300416e-01 3.29411409e-03 1.13996561e-04
 5.14401047e-02 1.05539529e-03 1.45806690e+01 5.59322453e-03
 2.10364839e+01 2.11336848e+00 1.04476133e-03 3.00150439e+00
 5.06829011e+00 1.95952297e+01 2.60120995e+01 5.18135786e-04
 7.25410483e-03 1.49928007e-02 3.40161046e+01 1.39610097e-04
 1.80218972e+01 2.51259180e-04 2.75750526e-03 7.76643316e+00
 3.01408330e-05 2.06596075e-03 8.27050639e-03 1.00990766e-02
 3.00149722e+00 1.96470363e-05 1.88060113e-02 3.01700473e-02
 3.99992452e+00 4.00011149e+00 2.86852821e-02 1.92937992e-03
 1.26179328e+00 4.95379265e-04 7.11118129e-03 4.55787670e-02
 2.83381018e-03 4.96769967e-05 1.27406586e-02 1.22288229e-02
 7.62846330e-04 2.00006369e+00 3.86194078e-02 2.91496917e-02
 1.50627648e-05 2.00074152e+00 3.62389446e-02 1.28595034e-03
 1.80265457e+01 2.00840145e+00 1.48383730e-01 4.29994400e+01
 5.24339069e-02 9.69218887e-04 2.80422007e-01 1.08493664e-03
 7.04563080e+00 1.22819413e+01 2.39991935e+01 4.32560983e-03
 1.46721303e+01 2.361211

In [16]:
print(x)

     LF_mtp_J__1   LF_mtp_J__2   LF_mtp_J__3   LF_mtp_J__4   LF_mtp_J__5  \
0   2.100060e-03  2.042049e-05  1.664535e-02  3.478520e-04  7.836108e-09   
1   1.639315e-02  3.783870e-02  9.768119e-05  7.510101e-05  7.085757e-02   
2   1.168348e-06  1.736744e-08  4.933058e-08  7.656540e-10  1.483258e-08   
3   9.359178e-09  4.477403e-06  4.395417e-08  2.063308e-06  5.876554e-05   
4   1.101619e-07  4.546296e-04  9.727482e-04  1.782762e-04  5.667455e-04   
5   2.652282e-07  9.907032e-08  4.819744e-10  3.225004e-12  4.797702e-05   
6   4.443956e-05  5.807227e-01  2.000000e+00  2.000000e+00  1.999908e+00   
7   7.585666e-07  2.875519e-07  3.732428e-04  2.517838e-04  3.658841e-03   
8   1.651035e-02  3.953820e-05  3.608287e-03  3.863653e-03  1.106629e-02   
9   3.022059e-02  1.472055e-04  2.949755e-01  1.750298e+00  3.488301e-02   
10  3.717974e-08  2.575776e-07  9.622872e-07  1.823141e-06  2.740771e-05   
11  1.959677e-06  3.756505e-08  1.159911e-04  2.999998e+00  1.422852e-06   
12  1.676465

In [16]:
true = pd.read_csv('../../train/training.csv')

In [39]:
mask = [patient_id in x['Patient_ID'].to_numpy() for patient_id in true['Patient_ID']]

true_part = true.iloc[mask]

In [30]:
cols = ['LF_mtp_J__1', 'LF_mtp_J__2', 'LF_mtp_J__3', 'LF_mtp_J__4','LF_mtp_J__5', 'LF_mtp_J__ip','RF_mtp_J__1','RF_mtp_J__2','RF_mtp_J__3', 'RF_mtp_J__4','RF_mtp_J__5','RF_mtp_J__ip']

pred = x[cols].to_numpy()
truth = true_part[cols].to_numpy()

In [36]:
np.sqrt(np.mean(np.square(pred - truth), axis = 0))

array([1.30853957, 1.20600466, 1.32900223, 1.19526512, 1.42064635,
       0.86388906, 1.38292799, 0.93861139, 1.35251965, 1.43232125,
       1.54085907, 1.03316863])

In [42]:
from train.train_joints_damage import _get_dataset

pretrained_base_model = tf.keras.models.load_model('feet_erosion_v1.h5', custom_objects=dependencies)

joint_dataset, tf_joint_dataset = _get_dataset(config, 'F', 'E')
    
outputs = []
for idx, class_weight in enumerate(joint_dataset.class_weights()):
    no_outcomes = len(class_weight.keys())
        
    output = Dense(no_outcomes, activation = 'softmax', name = f'output_{idx}')(pretrained_base_model)
    outputs.append(output)

model = keras.models.Model(
    inputs = pretrained_base_model.get_input_at(0),
    outputs = outputs,
    name = f'joint_model_{joint_type}_{dmg_type}')

FileNotFoundError: [Errno 2] File b'/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/training.csv' does not exist: b'/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/training_dataset/training.csv'